Reference:https://github.com/suragnair/alpha-zero-general/tree/master/othello

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [2]:

class Board:
    def __init__(self, n=4):
        self.n = n
        self.board = [[0] * n for _ in range(n)]
        self.current_play = 1
        self.reset()
        self.player1 = player1
        self.player2 = player2
        self.done = False
        self.directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

    def reset(self):
        n = self.n
        self.board = [[0] * n for _ in range(n)]
        self.current_play = 1
        board = self.board
        if n % 2 == 0:
            mid = n // 2
            board[mid - 1][mid - 1] = 1
            board[mid][mid] = 1
            board[mid - 1][mid] = -1
            board[mid][mid - 1] = -1

    def __getitem__(self, index):
        return self.board[index]

    def countDiff(self, color):
        count = 0
        for j in range(self.n):
            for i in range(self.n):
                if self[i][j] == color:
                    count += 1
                if self[i][j] == -color:
                    count -= 1
        return count

    def get_legal_moves(self, color):
        moves = set()  # stores the legal moves.
        for j in range(self.n):
            for i in range(self.n):
                if self[i][j] == color:
                    newmoves = self.get_moves_for_square((i, j))
                    moves.update(newmoves)
        return list(moves)

    def has_legal_moves(self, color):#check legal movel
        for j in range(self.n):
            for i in range(self.n):
                if self[i][j] == color:
                    newmoves = self.get_moves_for_square((i, j))
                    if len(newmoves) > 0:
                        return True
        return False

    def get_moves_for_square(self, square):
        (i, j) = square
        color = self[i][j]
        if color == 0:
            return None
        # search all possible directions.
        moves = []
        for direction in self.directions:
            move = self._discover_move(square, direction)
            if move:
                moves.append(move)
        # return the generated move list
        return moves

    def execute_move(self, move, color):
        flips = []
        for direction in self.directions:
            flip = self._get_flips(move, direction, color)
            flips.extend(flip)

        if flips:
            for i, j in flips:
                self[i][j] = color
        else:
            raise Exception("Invalid move")
        row, col = move
        self[row][col] = color
    #valid move in a specified direction starting from the origin position on the game board
    def _discover_move(self, origin, direction):
        i, j = origin
        color = self[i][j]
        flips = []
        for i, j in Board._increment_move(origin, direction, self.n):
            if self[i][j] == 0:
                if flips:
                    return (i, j)
                else:
                    return None
            elif self[i][j] == color:
                return None
            elif self[i][j] == -color:
                flips.append((i, j))
        return None

    def _get_flips(self, origin, direction, color):
        flips = []
        for i, j in Board._increment_move(origin, direction, self.n):
            if self[i][j] == 0:
                return []
            if self[i][j] == -color:
                flips.append((i, j))
            elif self[i][j] == color and len(flips) > 0:
                return flips
        return []
    #increment current position of the move inthe specified position
    def _increment_move(move, direction, n):
        i, j = move
        move = (i + direction[0], j + direction[1])#adding the next position
        while 0 <= move[0] < n and 0 <= move[1] < n:
            yield move
            move = (move[0] + direction[0], move[1] + direction[1])

    def step(self, action):
        if self.done:
            raise Exception("Game is already finished")
        legal_moves = self.get_legal_moves(self.current_play)
        if action not in legal_moves:
            raise Exception("Invalid move")
        self.execute_move(action, self.current_play)
        self.current_play = -self.current_play
        
        # Check if the game is over
        if not self.has_legal_moves(1) and not self.has_legal_moves(-1):
            self.done = True
        reward = 0 if not self.done else self.get_reward()
        return self.board, reward, self.done

    def get_reward(self):
        player1_score = self.countDiff(1)
        player2_score = self.countDiff(-1)
        if player1_score > player2_score:
            return 1  # Player 1 (Black) wins
        elif player1_score < player2_score:
            return -1  # Player 2 (White) wins
        else:
            return 0  # It's a draw

    def game_over(self):
        return not self.has_legal_moves(1) and not self.has_legal_moves(-1)
    
    #training 
    def play(self, player1, player2,total_games):
        random_wins1 = 0
        random_wins2 = 0
        draws = 0
        for _ in range(total_games):
            self.reset()  # Reset the board for each game.
            current_player = player1 if self.current_play == 1 else player2
            while True:
                self.print_board()
                if self.has_legal_moves(current_player.color):
                    print(f"{current_player.name}'s Turn")
                    move = current_player.get_move(self)
                    if move:
                        self.execute_move(move, current_player.color)
                        current_player = player1 if current_player == player2 else player2  
                else:
                    print("No legal moves for the current player.")
                    current_player = player1 if current_player == player2 else player2 

                if not (self.has_legal_moves(player1.color) or self.has_legal_moves(player2.color)):
                    break

            self.print_board()
            winner = self.get_reward()
            if winner == player1.color:
                random_wins1 += 1
                print(f"{player1.name} wins!")
            elif winner == player2.color:
                random_wins2 += 1
                print(f"{player2.name} wins!")
            else:
                draws += 1
                print("It's a draw!")

        print(f"{player1.name} wins: {random_wins1 / total_games:.2%}")
        print(f"{player2.name} wins: {random_wins2 / total_games:.2%}")
        print(f"Draws: {draws / total_games:.2%}")

    def print_board(self):
        for i in range(self.n):
            print('-----------------')
            print('|', end=' ')
            for j in range(self.n):
                if self.board[i][j] == 1:
                    print('B', end=' ')
                elif self.board[i][j] == -1:
                    print('W', end=' ')
                else:
                    print(' ', end=' ')
                print('|', end=' ')
            print()
        print('------------------')


In [3]:
## Random player

In [4]:

class Player:
    def __init__(self, color,name):
        self.color = color
        self.name= name

    def get_move(self, board):
        legal_moves = board.get_legal_moves(self.color)
        if legal_moves:
            return random.choice(legal_moves)
        return None

In [5]:
## Two random player plays against each other

In [6]:
if __name__ == "__main__":
    total_games=100 
    player1 = Player(1,"Random Player1")
    player2 = Player(-1,"Random Player2")    
    board = Board(n=4)
    board.play(board.player1, board.player2,total_games)


-----------------
|   |   |   |   | 
-----------------
|   | B | W |   | 
-----------------
|   | W | B |   | 
-----------------
|   |   |   |   | 
------------------
Random Player1's Turn
-----------------
|   |   |   |   | 
-----------------
|   | B | W |   | 
-----------------
| B | B | B |   | 
-----------------
|   |   |   |   | 
------------------
Random Player2's Turn
-----------------
|   |   |   |   | 
-----------------
| W | W | W |   | 
-----------------
| B | B | B |   | 
-----------------
|   |   |   |   | 
------------------
Random Player1's Turn
-----------------
|   |   | B |   | 
-----------------
| W | B | B |   | 
-----------------
| B | B | B |   | 
-----------------
|   |   |   |   | 
------------------
Random Player2's Turn
-----------------
|   |   | B |   | 
-----------------
| W | W | W | W | 
-----------------
| B | B | B |   | 
-----------------
|   |   |   |   | 
------------------
Random Player1's Turn
-----------------
| B |   | B |   | 
-----------------


|   | B | B |   | 
-----------------
|   | B |   |   | 
------------------
Random Player2's Turn
-----------------
|   |   |   |   | 
-----------------
|   | B | W |   | 
-----------------
|   | W | B |   | 
-----------------
| W | B |   |   | 
------------------
Random Player1's Turn
-----------------
|   |   | B |   | 
-----------------
|   | B | B |   | 
-----------------
|   | W | B |   | 
-----------------
| W | B |   |   | 
------------------
Random Player2's Turn
-----------------
|   |   | B | W | 
-----------------
|   | B | W |   | 
-----------------
|   | W | B |   | 
-----------------
| W | B |   |   | 
------------------
Random Player1's Turn
-----------------
|   |   | B | W | 
-----------------
|   | B | B | B | 
-----------------
|   | W | B |   | 
-----------------
| W | B |   |   | 
------------------
Random Player2's Turn
-----------------
|   |   | B | W | 
-----------------
|   | B | B | B | 
-----------------
|   | W | B |   | 
-----------------
| W | W | W |   | 

In [7]:
class HumanPlayer:
    def __init__(self, color, name):
        self.color = color
        self.name = name

    def get_move(self, board):
        while True:
            move = input(f"Enter your move : ")
            move = move.strip().upper()  # Convert to uppercase and remove extra whitespace
            if len(move) == 2 and move[0].isalpha() and move[1].isdigit():
                col = ord(move[0]) - ord('A')
                row = int(move[1]) - 1
                if (row, col) in board.get_legal_moves(self.color):
                    return (row, col)
                else:
                    print("Invalid move")
            else:
                print("Invalid input format.")



In [8]:
## One human player and other random player plays against each other

In [9]:
if __name__ == "__main__":
    player1 = Player(1," Random Player")
    
    player2 = HumanPlayer(-1,"Human Player")
    
    board = Board(n=4)
    
    board.play(board.player1, player2,1)


-----------------
|   |   |   |   | 
-----------------
|   | B | W |   | 
-----------------
|   | W | B |   | 
-----------------
|   |   |   |   | 
------------------
 Random Player's Turn
-----------------
|   |   | B |   | 
-----------------
|   | B | B |   | 
-----------------
|   | W | B |   | 
-----------------
|   |   |   |   | 
------------------
Human Player's Turn
Enter your move : b1
-----------------
|   | W | B |   | 
-----------------
|   | W | B |   | 
-----------------
|   | W | B |   | 
-----------------
|   |   |   |   | 
------------------
 Random Player's Turn
-----------------
|   | W | B |   | 
-----------------
| B | B | B |   | 
-----------------
|   | W | B |   | 
-----------------
|   |   |   |   | 
------------------
Human Player's Turn
Enter your move : d1
-----------------
|   | W | W | W | 
-----------------
| B | B | W |   | 
-----------------
|   | W | B |   | 
-----------------
|   |   |   |   | 
------------------
 Random Player's Turn
-----------------